In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
# Load the trained model
model = load_model('churn_model.h5')

# Load the encoder and scaler
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('ohe_geo.pkl', 'rb') as file:
    label_encoder_geo = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [3]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [10]:
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]])
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))

c:\Users\Sam Ben-Yosef\Desktop\ANN\annvenv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [8]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [14]:
input_data = pd.DataFrame([input_data])
input_data = pd.concat([input_data.reset_index(drop=True), geo_encoded_df], axis=1)

In [15]:
input_data 

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [16]:
input_data.dtypes

CreditScore            int64
Geography             object
Gender                object
Age                    int64
Tenure                 int64
Balance                int64
NumOfProducts          int64
HasCrCard              int64
IsActiveMember         int64
EstimatedSalary        int64
Geography_France     float64
Geography_Germany    float64
Geography_Spain      float64
dtype: object

In [19]:
# encode categorical variables
input_data['Gender'] = label_encoder_gender.transform(['Male'])
input_data 

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [21]:
# concatt one hot encoded data 
input_data = pd.concat([input_data.drop(columns=['Geography']), geo_encoded_df], axis=1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0,1.0,0.0,0.0


In [25]:
# Scaling the input data
# Ensure correct column order (remove duplicates and order as expected during training)
expected_columns = ['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Geography_France', 'Geography_Germany', 'Geography_Spain']
# Remove duplicate columns by keeping only the first occurrence
input_data_clean = input_data.loc[:, ~input_data.columns.duplicated()]
input_data_scaled = scaler.transform(input_data_clean[expected_columns])
input_data_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [26]:
# Predict churn
prediction = model.predict(input_data_scaled)
prediction 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step


array([[0.05953557]], dtype=float32)

In [27]:
prediction_probability = prediction[0][0]

In [28]:
prediction_probability

np.float32(0.059535567)

In [29]:
if prediction_probability > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is unlikely to churn.")

The customer is unlikely to churn.
